In [1]:
import pandas as pd

In [2]:
instancesHeader = ['GRAPH', 'SOURCE', 'TARGET']

instancesDf = pd.read_csv('netMakerInstances.inst', delimiter=' ', header=None, names=instancesHeader)
instancesDf['GRAPH'] = instancesDf['GRAPH'].apply(lambda x : x.split('/')[-1])
instancesDf

,GRAPH,SOURCE,TARGET
0,netMaker_3o_10000n_1amin_10amax_20i_1s_scorr_7...,8326,5528
1,netMaker_3o_10000n_1amin_10amax_20i_1s_scorr_7...,4543,5005
2,netMaker_3o_10000n_1amin_10amax_20i_1s_scorr_7...,3311,1812
3,netMaker_3o_10000n_1amin_10amax_20i_1s_scorr_7...,6160,6000
4,netMaker_3o_10000n_1amin_10amax_20i_1s_scorr_7...,6416,5350
...,...,...,...
1375,netMaker_3o_5000n_5amin_40amax_50i_1s_scorr_52...,3542,1356
1376,netMaker_3o_5000n_5amin_40amax_50i_1s_scorr_52...,2908,3284
1377,netMaker_3o_5000n_5amin_40amax_50i_1s_scorr_52...,4683,978
1378,netMaker_3o_5000n_5amin_40amax_50i_1s_scorr_52...,4627,2968


In [3]:
resultsHeader = ['ALGO', 'HOST' , 'GRAPH', 'NODES', 'ARCS' ,'SOURCE', 'TARGET', 'ITERATIONS', 'EXTRACTIONS', 'TARGET_SOLUTIONS', 'TIME', 'MEMORY', 'MAX_HEAP_SIZE']

mdaDf = pd.read_csv('mda_netMaker_3d.csv', delimiter=';', header=None, names=resultsHeader)
martinsDf = pd.read_csv('martins_netMaker_3d.csv', delimiter=';', header=None, names=resultsHeader)

mergedDf = pd.merge(instancesDf, mdaDf, on=['GRAPH','SOURCE', 'TARGET'], how='left')
mergedDf = pd.merge(mergedDf, martinsDf, on=['GRAPH', 'SOURCE', 'TARGET', 'NODES', 'ARCS'], how='left', suffixes=("_mda", "_martins"))
mergedDf
mergedDf.loc[mergedDf['TIME_martins']<0.0001, 'TIME_martins'] = 0.0001
mergedDf.loc[mergedDf['TIME_mda']<0.0001, 'TIME_mda'] = 0.0001

In [4]:
mergedDf['SPEEDUP'] = mergedDf['TIME_martins']/mergedDf['TIME_mda']
mergedDf['DENSITY'] = mergedDf['ARCS']/mergedDf['NODES']
mergedDf['DENSITY'] = mergedDf['DENSITY'].apply(lambda x: round(x))
mergedDf['LABELS_PER_SEC_mda'] = mergedDf['EXTRACTIONS_mda']/mergedDf['TIME_mda']
mergedDf['LABELS_PER_SEC_martins'] = mergedDf['EXTRACTIONS_martins']/mergedDf['TIME_martins']

# Density Analysis
Analyze the performance loss of both algorithms as the densities of the graphs increase.

In [8]:
from scipy.stats import gmean
densities = []
densitySpeed = []
densitySpeed_martins = []
densitySpeedup = []
for density, densityDf in mergedDf.groupby(by='DENSITY'):
    print(density, densityDf['NODES'].mean(), len(densityDf))
    densities.append(density)
    densitySpeed.append(gmean(densityDf['LABELS_PER_SEC_mda']))
    densitySpeed_martins.append(gmean(densityDf['LABELS_PER_SEC_martins']))
    densitySpeedup.append(gmean(densityDf['SPEEDUP']))

# print(densities)
# print(densitySpeed)
# print(densitySpeed_martins)
import matplotlib.pyplot as plt

c1 = '#FFC857'
c2 = '#255F85'

mda = list(mergedDf[mergedDf['TIME_mda'].notna()]['LABELS_PER_SEC_mda'])
xmda = list(mergedDf['DENSITY'])
martins = list(mergedDf[mergedDf['TIME_martins'].notna()]['TIME_martins'])
xmartins = list(mergedDf['EXTRACTIONS_martins'])
fig = plt.figure()
fig.patch.set_alpha(0.0)
ax1 = fig.add_subplot(111)

ax1.scatter(densities, densitySpeed, s=10, c=c2, marker="s", label='MDA')
ax1.scatter(densities, densitySpeed_martins, s=10, c=c1, marker="o", label='Martins')
ax1.set_xticks(densities)
#ax1.set_yscale('log')
ax1.yaxis.grid(True)
plt.ylabel('Permanent labels per second')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Densities of the NetMaker graphs rounded to the next integer')
plt.legend()
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Average speedup')  # we already handled the x-label with ax1
ax2.scatter(densities, densitySpeedup, s=10, c='red', marker="o", label='Speedup')
ax2.tick_params(axis='y')
ax2.legend(loc='center right')

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title(f'Grids 3d')
plt.savefig(f'./plots/netmaker_3d_martins_mda_densities.svg')
#plt.show()
plt.close()

6 17500.0 240
8 17500.0 240
11 16818.18181818182 220
13 17500.0 240
16 16000.0 100
17 17500.0 120
21 19000.0 100
23 17500.0 120


In [33]:
from scipy.stats import gmean
#extractionLimits = [10**i for i in range (9)]
#extractionLimits
print(f"Density & Instances & avg. $N$ & avg $N_t$ & avg. time mda & avg. time martins & Speedup\\\\")
for density, slicedDf in mergedDf.groupby(by='DENSITY'):
    #print(len(slicedDf), slicedDf['TIME_martins'])
    gmean_mda = gmean(slicedDf.loc[slicedDf['TIME_mda']>=0.0001]['TIME_mda'])
    gmean_martins = gmean(slicedDf.loc[slicedDf['TIME_martins']>=0.0001]['TIME_martins'])
    #gmeanSpeedup = gmean(slicedDf.loc[slicedDf['TIME_martins']>0.0001]['TIME_martins']/slicedDf.loc[slicedDf['TIME_mda']>0.0001]['TIME_mda'])
    gmeanSpeedup = gmean_martins/gmean_mda
    print(f"{density} ")
    print(f"& {len(slicedDf)} & {gmean(slicedDf['EXTRACTIONS_mda']):.2f} & {gmean(slicedDf['TARGET_SOLUTIONS_mda']):.2f}")
    print(f"& {gmean_mda:.4f} & {gmean_martins:.4f}")
    print(f"& {gmeanSpeedup:.2f}\\\\\n")

Nodes & Instances & avg. $N$ & avg $N_t$ & avg. time mda & avg. time martins & Speedup\\
6 
& 240 & 1927527.91 & 319.27
& 7.1516 & 8.7192
& 1.22\\

8 
& 240 & 3388611.54 & 605.15
& 20.3446 & 28.1831
& 1.39\\

11 
& 220 & 5409865.60 & 994.40
& 52.4636 & 81.0655
& 1.55\\

13 
& 240 & 7310322.13 & 1311.05
& 95.0760 & 154.1649
& 1.62\\

16 
& 100 & 14126390.82 & 2905.09
& 299.7726 & 591.6235
& 1.97\\

17 
& 120 & 19265540.88 & 3362.51
& 531.3614 & 1026.5097
& 1.93\\

21 
& 100 & 11341181.08 & 1975.62
& 258.5909 & 439.6957
& 1.70\\

23 
& 120 & 11478804.80 & 2070.82
& 289.8834 & 505.3561
& 1.74\\



In [20]:
for nodes, nodesDf in mergedDf.groupby(by='NODES'):
    fig = plt.figure()
    fig.patch.set_alpha(0.0)
    ax1 = fig.add_subplot(111)

    ax1.scatter(nodesDf['EXTRACTIONS_mda'], nodesDf['TIME_mda'], s=10, c=c2, marker="s", label='MDA')
    ax1.scatter(nodesDf['EXTRACTIONS_mda'], nodesDf['TIME_martins'], s=10, c=c1, marker="o", label='Martins')
    #ax1.set_xticks(densities)
    #ax1.set_yscale('log')
    ax1.yaxis.grid(True)
    plt.ylabel('Time [s]')
    #plt.xscale('log')
    #plt.yscale('log')
    plt.xlabel('Number $N$ of permanent paths')
    plt.legend()
    plt.title(f'Netmaker 3d graphs with {nodes} nodes')
    plt.savefig(f'./plots/netmaker_3d_martins_mda_time_{nodes}.svg')
    plt.close()
    #plt.show()

In [21]:
from scipy.stats import gmean
#extractionLimits = [10**i for i in range (9)]
#extractionLimits
print(f"Nodes & Instances & avg. $N$ & avg $N_t$ & avg. time mda & avg. time martins & Speedup\\\\")
for nodes, slicedDf in mergedDf.groupby(by='NODES'):
    #print(len(slicedDf), slicedDf['TIME_martins'])
    gmean_mda = gmean(slicedDf.loc[slicedDf['TIME_mda']>0.0001]['TIME_mda'])
    gmean_martins = gmean(slicedDf.loc[slicedDf['TIME_martins']>0.0001]['TIME_martins'])
    #gmeanSpeedup = gmean(slicedDf.loc[slicedDf['TIME_martins']>0.0001]['TIME_martins']/slicedDf.loc[slicedDf['TIME_mda']>0.0001]['TIME_mda'])
    gmeanSpeedup = gmean_martins/gmean_mda
    print(f"{nodes} ")
    print(f"& {len(slicedDf)} & {gmean(slicedDf['EXTRACTIONS_mda']):.2f} & {gmean(slicedDf['TARGET_SOLUTIONS_mda']):.2f}")
    print(f"& {gmean_mda:.4f} & {gmean_martins:.4f}")
    print(f"& {gmeanSpeedup:.2f}\\\\\n")

Nodes & Instances & avg. $N$ & avg $N_t$ & avg. time mda & avg. time martins & Speedup\\
5000 
& 240 & 1197681.03 & 695.66
& 6.6550 & 12.1276
& 1.82\\

10000 
& 220 & 3398739.57 & 941.42
& 30.7094 & 48.0459
& 1.56\\

15000 
& 240 & 6639691.63 & 1126.15
& 78.3546 & 122.1306
& 1.56\\

20000 
& 240 & 9868444.80 & 1252.33
& 132.7924 & 204.8363
& 1.54\\

25000 
& 200 & 11313273.81 & 1125.89
& 153.4810 & 227.2383
& 1.48\\

30000 
& 240 & 15213764.84 & 1344.04
& 230.8244 & 346.3387
& 1.50\\



In [73]:
mergedDf[mergedDf['GRAPH'].str.contains('CTR')]

,GRAPH,SOURCE,TARGET,K,ALGO_na,TIME_na,COST_MIN_na,COST_MAX_na,SOLUTIONS_na,BDA_QUERIES,...,NODES,ARCS,SOLUTIONS_kurz,COST_MIN_kurz,COST_MAX_kurz,SUM_OF_COSTS,SP_TREES,SP_TREES_EXPANSIONS,TIME_kurz,SPEEDUP
5400,USA-road-dt.CTR.gr,12181763,10663437,10,NaN,7200.0,NaN,NaN,NaN,NaN,...,14081816.0,34292496.0,10.0,1268594.0,1268594.0,12685940.0,1.0,168979.0,1.627,0.000226
5401,USA-road-dt.CTR.gr,13263609,3512051,10,NaN,7200.0,NaN,NaN,NaN,NaN,...,14081816.0,34292496.0,10.0,5474977.0,5474977.0,54749770.0,1.0,3856891.0,3.875,0.000538
5402,USA-road-dt.CTR.gr,8617129,2921811,10,NaN,7200.0,NaN,NaN,NaN,NaN,...,14081816.0,34292496.0,10.0,8888964.0,8888964.0,88889640.0,1.0,7897146.0,7.238,0.001005
5403,USA-road-dt.CTR.gr,13145916,309892,10,NaN,7200.0,NaN,NaN,NaN,NaN,...,14081816.0,34292496.0,10.0,12879745.0,12879745.0,128797450.0,1.0,10634038.0,8.625,0.001198
5404,USA-road-dt.CTR.gr,10923367,8213275,10,NaN,7200.0,NaN,NaN,NaN,NaN,...,14081816.0,34292496.0,10.0,3839849.0,3839849.0,38398490.0,1.0,1862878.0,1.920,0.000267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,USA-road-dt.CTR.gr,628005,7120819,1000000,NaN,7200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200.000,1.000000
7196,USA-road-dt.CTR.gr,7228136,11312825,1000000,NaN,7200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200.000,1.000000
7197,USA-road-dt.CTR.gr,4648576,7370601,1000000,NaN,7200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200.000,1.000000
7198,USA-road-dt.CTR.gr,9644393,10542969,1000000,NaN,7200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200.000,1.000000
